<a href="https://www.kaggle.com/code/rishabhbhartiya/dubai-aqi-2024-event-based-analysis?scriptVersionId=245441230" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import warnings
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation
from matplotlib import transforms
import matplotlib.patches as patches
import matplotlib.animation as animation
from matplotlib.patches import Circle, Polygon
warnings.filterwarnings('ignore')

df = pd.read_csv("/kaggle/input/air-quality-2024/Dubai_Air_Quality.csv")

def get_day_period(hour):
    if 5 <= hour < 12:
        return 'MORNING'
    elif 12 <= hour < 17:
        return 'AFTERNOON'
    elif 17 <= hour < 21:
        return 'EVENING'
    else:
        return 'NIGHT'

def label_weather(month):
    if month in [12, 1, 2]:
        return 'Mild'
    elif month in [3, 4, 5]:
        return 'Hot & Dusty'
    elif month in [6, 7, 8, 9]:
        return 'Extreme Heat & Humid'
    else:  # 10, 11
        return 'Warm & Dry'

def get_aqi_category_and_indicator(aqi):
    if 0 <= aqi <= 50:
        return "Good", "Green"
    elif 51 <= aqi <= 100:
        return "Moderate", "Yellow"
    elif 101 <= aqi <= 150:
        return "Unhealthy for Sensitive Groups", "Orange"
    elif 151 <= aqi <= 200:
        return "Unhealthy", "Red"
    elif 201 <= aqi <= 300:
        return "Very Unhealthy", "Purple"
    elif 301 <= aqi <= 500:
        return "Hazardous", "Maroon"
    else:
        return "AQI out of range", "Unknown"

df['AQI'] = df['AQI'].round().astype(int)
df['DATETIME'] = pd.to_datetime(df['Date'])         
df['DATE'] = df['DATETIME'].dt.date                 
df['TIME'] = df['DATETIME'].dt.time.astype(str).str.slice(0, 5) 
df['HOUR'] = df['DATETIME'].dt.hour
df['AM_PM'] = df['HOUR'].apply(lambda x: 'AM' if x < 12 else 'PM')
df['DAY_PERIOD'] = df['HOUR'].apply(get_day_period)
df['WEATHER_LABEL'] = df['DATETIME'].dt.month.apply(label_weather)
df[['CATEGORY', 'INDICATOR']] = df['AQI'].apply(lambda x: pd.Series(get_aqi_category_and_indicator(x)))


df = df.drop(columns = ["CO2", "Date"])

In [2]:
df_day = df[df['DATE'] == pd.to_datetime("2024-04-10").date()].sort_values(by='HOUR').reset_index(drop=True)
pollutant_max_values = {"PM2.5": 250, "PM10": 430, "CO": 15000, "SO2": 200, "NO2": 200, "O3": 200}
fig, ax = plt.subplots(figsize=(16, 9)) 
fig.subplots_adjust(left=0.03, right=0.97, top=0.95, bottom=0.05) 

def draw_frame(i):
    ax.clear()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')

    gradient = np.linspace(0.1, 0.2, 256)
    ax.imshow(np.vstack((gradient, gradient)), cmap='viridis', extent=[0, 1, 0, 1], alpha=0.9, aspect='auto')
    fig.patch.set_facecolor('#1a1a1a')

    row = df_day.iloc[i]
    ax.text(0.05, 0.85, "Air Quality Index (AQI)", fontsize=24, color='white', weight='bold', alpha=0.8)
    ax.text(0.05, 0.65, f"{row['AQI']}", fontsize=80, color='gold', weight='bold')
    ax.text(0.05, 0.53, f"{row['CATEGORY']}", fontsize=22, color='white', weight='bold')
    
    metadata_text = (
    f"Date: {row['DATE'].strftime('%B %d, %Y')}\n"
    f"Time: {row['TIME']} {row['AM_PM']} ({row['DAY_PERIOD']})\n"
    f"Weather: {row['WEATHER_LABEL']}")
    ax.text(0.68, 0.88, metadata_text, fontsize=15, color='white', va='top',
        ha='left', linespacing=1.6, alpha=0.9, weight='bold')

# Clock 
    clock_ax = fig.add_axes([0.37, 0.65, 0.26, 0.26])  # [left, bottom, width, height] in figure coords
    clock_ax.set_aspect('equal')
    clock_ax.axis('off')
    clock_ax.add_patch(Circle((0.5, 0.5), 0.48, fc='#2a2a2a', ec='cyan', lw=2))
    clock_ax.add_patch(Circle((0.5, 0.5), 0.015, fc='white', zorder=10))
    for minute in range(60):
        angle = np.deg2rad(90 - minute * 6)
        is_hour = minute % 5 == 0
        length = 0.08 if is_hour else 0.03
        color = 'white' if is_hour else 'grey'
        start = 0.5 + (0.4 - length) * np.array([np.cos(angle), np.sin(angle)])
        end   = 0.5 + 0.4 * np.array([np.cos(angle), np.sin(angle)])
        clock_ax.plot([start[0], end[0]], [start[1], end[1]], color=color, lw=2 if is_hour else 1)

        if is_hour and minute > 0:
            hour_num = minute // 5
            clock_ax.text(0.5 + 0.32 * np.cos(angle), 0.5 + 0.32 * np.sin(angle),
                          str(hour_num), ha='center', va='center', fontsize=10, color='white')
    hour = row['HOUR'] % 12
    minute = int(row['TIME'].split(":")[1])
    hour_angle = np.deg2rad(90 - (hour * 30 + minute * 0.5))
    minute_angle = np.deg2rad(90 - minute * 6)
    clock_ax.plot([0.5, 0.5 + 0.2 * np.cos(hour_angle)],
              [0.5, 0.5 + 0.2 * np.sin(hour_angle)],
              color='white', lw=4)
    clock_ax.plot([0.5, 0.5 + 0.35 * np.cos(minute_angle)],
              [0.5, 0.5 + 0.35 * np.sin(minute_angle)],
              color='white', lw=2)
    clock_ax.plot([0.5, 0.5], [0.5, 0.92], color='red', lw=1)

# Pollutant Cards
    pollutants = ["PM2.5", "PM10", "O3", "NO2", "SO2", "CO"]
    colors = ['magenta', 'orange', 'cyan', 'violet', 'lightgreen', 'lightblue']

    card_w, card_h = 0.18, 0.16
    x_start, y_start = 0.15, 0.30
    x_spacing, y_spacing = 0.26, 0.22

    for j, (pollutant, color) in enumerate(zip(pollutants, colors)):
        col = j % 3
        row_idx = j // 3
        x = x_start + col * x_spacing
        y = y_start - row_idx * y_spacing

        ax.add_patch(patches.FancyBboxPatch((x, y), card_w, card_h,
                                            boxstyle="round,pad=0.02", fc='#2a2a2a', ec=color, lw=1.5))
        unit = "µg/m³" if "PM" in pollutant or pollutant in ["O3", "NO2", "SO2"] else "ppb"
        ax.text(x + 0.015, y + card_h - 0.04, pollutant, fontsize=16, color=color, weight='bold')
        ax.text(x + 0.015, y + card_h - 0.08, f"{row[pollutant]:.1f}", fontsize=18, color='white')
        ax.text(x + 0.09, y + card_h - 0.08, unit, fontsize=12, color='white', alpha=0.7)

        bar_x, bar_y = x + 0.015, y + 0.02
        bar_w, bar_h = card_w - 0.03, 0.02
        bar_fill_width = (row[pollutant] / pollutant_max_values.get(pollutant, 1)) * bar_w
        ax.add_patch(patches.Rectangle((bar_x, bar_y), bar_w, bar_h, fc='#1a1a1a', ec='grey'))
        ax.add_patch(patches.Rectangle((bar_x, bar_y), bar_fill_width, bar_h, fc=color))

fig.tight_layout(pad=2.0)

ani = animation.FuncAnimation(fig, draw_frame, frames=len(df_day), interval=1000)
output_filename = "eid_al_fitr_dashboard.gif"
ani.save(output_filename, writer='pillow', fps=1, dpi=100)
plt.close(fig)

In [3]:
start_date = datetime(2024, 6, 16).date()
end_date = datetime(2024, 6, 20).date()
eid_al_adha_df = df[(df['DATE'] >= start_date) & (df['DATE'] <= end_date)]
df_day = eid_al_adha_df.sort_values(by=['DATE', 'HOUR']).reset_index(drop=True)


pollutant_max_values = {"PM2.5": 250, "PM10": 430, "CO": 15000, "SO2": 200, "NO2": 200, "O3": 200}
fig, ax = plt.subplots(figsize=(16, 9)) 
fig.subplots_adjust(left=0.03, right=0.97, top=0.95, bottom=0.05) 

def draw_frame(i):
    ax.clear()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')

    gradient = np.linspace(0.1, 0.2, 256)
    ax.imshow(np.vstack((gradient, gradient)), cmap='viridis', extent=[0, 1, 0, 1], alpha=0.9, aspect='auto')
    fig.patch.set_facecolor('#1a1a1a')
    row = df_day.iloc[i]
    ax.text(0.05, 0.85, "Air Quality Index (AQI)", fontsize=24, color='white', weight='bold', alpha=0.8)
    ax.text(0.05, 0.65, f"{row['AQI']}", fontsize=80, color='gold', weight='bold')
    ax.text(0.05, 0.53, f"{row['CATEGORY']}", fontsize=22, color='white', weight='bold')
    metadata_text = (
        f"Date: {row['DATE'].strftime('%B %d, %Y')}\n"
        f"Day: {row['DATE'].strftime('%A')}\n"
        f"Time: {row['TIME']} {row['AM_PM']} ({row['DAY_PERIOD']})\n"
        f"Weather: {row['WEATHER_LABEL']}"
    )
    ax.text(0.68, 0.88, metadata_text, fontsize=15, color='white', va='top',
            ha='left', linespacing=1.6, alpha=0.9, weight='bold')

    # Clock
    clock_ax = fig.add_axes([0.37, 0.65, 0.26, 0.26])
    clock_ax.set_aspect('equal')
    clock_ax.axis('off')
    clock_ax.add_patch(Circle((0.5, 0.5), 0.48, fc='#2a2a2a', ec='cyan', lw=2))
    clock_ax.add_patch(Circle((0.5, 0.5), 0.015, fc='white', zorder=10))
    for minute in range(60):
        angle = np.deg2rad(90 - minute * 6)
        is_hour = minute % 5 == 0
        length = 0.08 if is_hour else 0.03
        color = 'white' if is_hour else 'grey'
        start = 0.5 + (0.4 - length) * np.array([np.cos(angle), np.sin(angle)])
        end   = 0.5 + 0.4 * np.array([np.cos(angle), np.sin(angle)])
        clock_ax.plot([start[0], end[0]], [start[1], end[1]], color=color, lw=2 if is_hour else 1)

        if is_hour and minute > 0:
            hour_num = minute // 5
            clock_ax.text(0.5 + 0.32 * np.cos(angle), 0.5 + 0.32 * np.sin(angle),
                          str(hour_num), ha='center', va='center', fontsize=10, color='white')

    hour = row['HOUR'] % 12
    minute = int(row['TIME'].split(":")[1])
    hour_angle = np.deg2rad(90 - (hour * 30 + minute * 0.5))
    minute_angle = np.deg2rad(90 - minute * 6)
    clock_ax.plot([0.5, 0.5 + 0.2 * np.cos(hour_angle)],
                  [0.5, 0.5 + 0.2 * np.sin(hour_angle)],
                  color='white', lw=4)
    clock_ax.plot([0.5, 0.5 + 0.35 * np.cos(minute_angle)],
                  [0.5, 0.5 + 0.35 * np.sin(minute_angle)],
                  color='white', lw=2)
    clock_ax.plot([0.5, 0.5], [0.5, 0.92], color='red', lw=1)

    # Pollutant Cards
    pollutants = ["PM2.5", "PM10", "O3", "NO2", "SO2", "CO"]
    colors = ['magenta', 'orange', 'cyan', 'violet', 'lightgreen', 'lightblue']
    card_w, card_h = 0.18, 0.16
    x_start, y_start = 0.15, 0.30
    x_spacing, y_spacing = 0.26, 0.22

    for j, (pollutant, color) in enumerate(zip(pollutants, colors)):
        col = j % 3
        row_idx = j // 3
        x = x_start + col * x_spacing
        y = y_start - row_idx * y_spacing

        ax.add_patch(patches.FancyBboxPatch((x, y), card_w, card_h,
                                            boxstyle="round,pad=0.02", fc='#2a2a2a', ec=color, lw=1.5))
        unit = "µg/m³" if "PM" in pollutant or pollutant in ["O3", "NO2", "SO2"] else "ppb"
        ax.text(x + 0.015, y + card_h - 0.04, pollutant, fontsize=16, color=color, weight='bold')
        ax.text(x + 0.015, y + card_h - 0.08, f"{row[pollutant]:.1f}", fontsize=18, color='white')
        ax.text(x + 0.09, y + card_h - 0.08, unit, fontsize=12, color='white', alpha=0.7)

        bar_x, bar_y = x + 0.015, y + 0.02
        bar_w, bar_h = card_w - 0.03, 0.02
        bar_fill_width = (row[pollutant] / pollutant_max_values.get(pollutant, 1)) * bar_w
        ax.add_patch(patches.Rectangle((bar_x, bar_y), bar_w, bar_h, fc='#1a1a1a', ec='grey'))
        ax.add_patch(patches.Rectangle((bar_x, bar_y), bar_fill_width, bar_h, fc=color))

fig.tight_layout(pad=2.0)
ani = animation.FuncAnimation(fig, draw_frame, frames=len(df_day), interval=1000)

output_filename = "eid_al_adha_aqi_dashboard.gif"
ani.save(output_filename, writer='pillow', fps=1, dpi=100)
plt.close(fig)

In [4]:
df_day = df[df['DATE'] == pd.to_datetime("2024-12-31").date()].sort_values(by='HOUR').reset_index(drop=True)
pollutant_max_values = {"PM2.5": 250, "PM10": 430, "CO": 15000, "SO2": 200, "NO2": 200, "O3": 200}
fig, ax = plt.subplots(figsize=(16, 9)) 
fig.subplots_adjust(left=0.03, right=0.97, top=0.95, bottom=0.05) 

def draw_frame(i):
    ax.clear()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')

    gradient = np.linspace(0.1, 0.2, 256)
    ax.imshow(np.vstack((gradient, gradient)), cmap='viridis', extent=[0, 1, 0, 1], alpha=0.9, aspect='auto')
    fig.patch.set_facecolor('#1a1a1a')

    row = df_day.iloc[i]
    ax.text(0.05, 0.85, "Air Quality Index (AQI)", fontsize=24, color='white', weight='bold', alpha=0.8)
    ax.text(0.05, 0.65, f"{row['AQI']}", fontsize=80, color='gold', weight='bold')
    ax.text(0.05, 0.53, f"{row['CATEGORY']}", fontsize=22, color='white', weight='bold')
    
    metadata_text = (
    f"Date: {row['DATE'].strftime('%B %d, %Y')}\n"
    f"Time: {row['TIME']} {row['AM_PM']} ({row['DAY_PERIOD']})\n"
    f"Weather: {row['WEATHER_LABEL']}")
    ax.text(0.68, 0.88, metadata_text, fontsize=15, color='white', va='top',
        ha='left', linespacing=1.6, alpha=0.9, weight='bold')

# Clock 
    clock_ax = fig.add_axes([0.37, 0.65, 0.26, 0.26])  # [left, bottom, width, height] in figure coords
    clock_ax.set_aspect('equal')
    clock_ax.axis('off')
    clock_ax.add_patch(Circle((0.5, 0.5), 0.48, fc='#2a2a2a', ec='cyan', lw=2))
    clock_ax.add_patch(Circle((0.5, 0.5), 0.015, fc='white', zorder=10))
    for minute in range(60):
        angle = np.deg2rad(90 - minute * 6)
        is_hour = minute % 5 == 0
        length = 0.08 if is_hour else 0.03
        color = 'white' if is_hour else 'grey'
        start = 0.5 + (0.4 - length) * np.array([np.cos(angle), np.sin(angle)])
        end   = 0.5 + 0.4 * np.array([np.cos(angle), np.sin(angle)])
        clock_ax.plot([start[0], end[0]], [start[1], end[1]], color=color, lw=2 if is_hour else 1)

        if is_hour and minute > 0:
            hour_num = minute // 5
            clock_ax.text(0.5 + 0.32 * np.cos(angle), 0.5 + 0.32 * np.sin(angle),
                          str(hour_num), ha='center', va='center', fontsize=10, color='white')
    hour = row['HOUR'] % 12
    minute = int(row['TIME'].split(":")[1])
    hour_angle = np.deg2rad(90 - (hour * 30 + minute * 0.5))
    minute_angle = np.deg2rad(90 - minute * 6)
    clock_ax.plot([0.5, 0.5 + 0.2 * np.cos(hour_angle)],
              [0.5, 0.5 + 0.2 * np.sin(hour_angle)],
              color='white', lw=4)
    clock_ax.plot([0.5, 0.5 + 0.35 * np.cos(minute_angle)],
              [0.5, 0.5 + 0.35 * np.sin(minute_angle)],
              color='white', lw=2)
    clock_ax.plot([0.5, 0.5], [0.5, 0.92], color='red', lw=1)

# Pollutant Cards
    pollutants = ["PM2.5", "PM10", "O3", "NO2", "SO2", "CO"]
    colors = ['magenta', 'orange', 'cyan', 'violet', 'lightgreen', 'lightblue']

    card_w, card_h = 0.18, 0.16
    x_start, y_start = 0.15, 0.30
    x_spacing, y_spacing = 0.26, 0.22

    for j, (pollutant, color) in enumerate(zip(pollutants, colors)):
        col = j % 3
        row_idx = j // 3
        x = x_start + col * x_spacing
        y = y_start - row_idx * y_spacing

        ax.add_patch(patches.FancyBboxPatch((x, y), card_w, card_h,
                                            boxstyle="round,pad=0.02", fc='#2a2a2a', ec=color, lw=1.5))
        unit = "µg/m³" if "PM" in pollutant or pollutant in ["O3", "NO2", "SO2"] else "ppb"
        ax.text(x + 0.015, y + card_h - 0.04, pollutant, fontsize=16, color=color, weight='bold')
        ax.text(x + 0.015, y + card_h - 0.08, f"{row[pollutant]:.1f}", fontsize=18, color='white')
        ax.text(x + 0.09, y + card_h - 0.08, unit, fontsize=12, color='white', alpha=0.7)

        bar_x, bar_y = x + 0.015, y + 0.02
        bar_w, bar_h = card_w - 0.03, 0.02
        bar_fill_width = (row[pollutant] / pollutant_max_values.get(pollutant, 1)) * bar_w
        ax.add_patch(patches.Rectangle((bar_x, bar_y), bar_w, bar_h, fc='#1a1a1a', ec='grey'))
        ax.add_patch(patches.Rectangle((bar_x, bar_y), bar_fill_width, bar_h, fc=color))

fig.tight_layout(pad=2.0)

ani = animation.FuncAnimation(fig, draw_frame, frames=len(df_day), interval=1000)
output_filename = "new_year_eve_dashboard.gif"
ani.save(output_filename, writer='pillow', fps=1, dpi=100)
plt.close(fig)

<div style="
    width: 700px;
    padding: 30px;
    border-radius: 20px;
    background: #e0e0e0;
    box-shadow: 15px 15px 30px #bebebe, -15px -15px 30px #ffffff;
    display: flex;
    align-items: center;
    justify-content: center;
    margin: 20px auto;
">
  <img src="https://raw.githubusercontent.com/rishabhbhartiya/KAGGLE/3d7f0f36726937f2397b806fc3b60f649dd5405b/ChatGPT%20Image%20Jun%2010%2C%202025%20at%2011_46_48%20AM.png" alt="f" style="border-radius: 15px; max-width: 100%; height: auto;" />
</div>


<div style="
    background-color: #f5f5f5;
    border: 2px solid #555;
    padding: 20px;
    border-radius: 8px;
    font-family: 'Georgia', serif;
    line-height: 1.8;
    max-width: 700px;
    margin: auto;
    box-shadow: 5px 5px 10px rgba(0,0,0,0.1);">
    <h2 style="color: #333; text-align: center;">EVENT BASED ANALYSIS</h2>
    <p>🕌 Religious Events</p>
    <blockquote style="border-left: 4px solid #555; padding-left: 10px;">
        Eid al-Fitr (April 10) – potential spikes in traffic & firework-related pollution.
    </blockquote>
    <blockquote style="border-left: 4px solid #555; padding-left: 10px;">
        Eid al-Adha (June 16–20) – extended holidays can alter traffic patterns.
    </blockquote><br>
    <p>🎉 Commercial & National Events</p>
    <blockquote style="border-left: 4px solid #555; padding-left: 10px;">
        New Year's Eve (Dec 31) – fireworks and celebrations = pollution peak.
    </blockquote><br>

<!-- Sleek Outline Banner -->
<div style="
    width: 100%;
    max-width: 700px;
    margin: 20px auto;
    padding: 20px;
    border-radius: 10px;
    background: #f5f5f5;
    color: #222;
    font-family: 'Courier New', monospace;
    font-size: 18px;
    font-weight: bold;
    text-align: center;
    text-transform: uppercase;
    letter-spacing: 2px;
    border: 2px solid #555;
">
    🌙 EID AL-FITR [10 APRIL 2024]
</div>

In [5]:
HTML('<img src="eid_al_fitr_dashboard.gif" width="800">')

<div style="
    width: 100%;
    max-width: 700px;
    margin: 20px auto;
    padding: 20px;
    border-radius: 10px;
    background: #f5f5f5;
    color: #222;
    font-family: 'Courier New', monospace;
    font-size: 18px;
    font-weight: bold;
    text-align: center;
    text-transform: uppercase;
    letter-spacing: 2px;
    border: 2px solid #555;
">
    🌙 EID AL-ADHA [16- 20 JUNE 2024]
</div>

In [6]:
HTML('<img src="eid_al_adha_aqi_dashboard.gif" width="800">')

<div style="
    width: 100%;
    max-width: 700px;
    margin: 20px auto;
    padding: 20px;
    border-radius: 10px;
    background: #f5f5f5;
    color: #222;
    font-family: 'Courier New', monospace;
    font-size: 18px;
    font-weight: bold;
    text-align: center;
    text-transform: uppercase;
    letter-spacing: 2px;
    border: 2px solid #555;
">
    NEW YEAR'S EVE [31 DECEMBER 2024]
</div>

In [7]:
HTML('<img src="new_year_eve_dashboard.gif" width="800">')

<div style="
    width: 100%;
    max-width: 700px;
    margin: 20px auto;
    padding: 20px;
    border-radius: 10px;
    background: #f5f5f5;
    color: #222;
    font-family: 'Courier New', monospace;
    font-size: 18px;
    font-weight: bold;
    text-align: center;
    text-transform: uppercase;
    letter-spacing: 2px;
    border: 2px solid #555;
">
    THANK YOU FOR VISITING THIS NOTEBOOK, PLEASE 🔼 UPVOTE IF YOU LIKE MY WORK.
</div>